# Use Numerov to solve the Harmonic Oscillator
In this example, we use atomic units. Mass unit is electron mass $m_e$=1. $\hbar=1$. Energy unit is Hartree $E_h$=1. Length unit is bohr.

Particle mass $m$ (in $m_e$), angular frequqnecy $\omega$ (unitless) will be input parameters of the NumerovSolverHO class.

$$\psi''=-\frac{2m}{\hbar^2}[E-V(x)]\psi$$

$$\psi(-\infty)=0$$

$$\psi(\infty)=0$$

where $$V(x) = \frac{1}{2}m\omega^2x^2$$

* Unlike the PIB example, here we are not going to assume that we know the energy. Rather, energy is an input parameter. We will give some guess energy $E$ and use Numerov to solve the corresponding wave function.

Based on Numerov method, Schrodinger equation of the following form

$$\psi''=-k^2(x) \psi$$

has the finite difference approximation

$$\psi(x_{n+1}) = 2\psi(x_n)-\psi(x_{n-1})-\delta^2k^2(x_n)\psi(x_n)$$

and in this case $k^2(x_n)=\frac{2m}{\hbar^2}[E-V(x)]$

Now it's straight forward to write the code similar to the PIB Numerov solver.

Some remaining questions:

**Q:** It's not practical to propogate from $x=-\infty$. Which region should we choose to set the grids?

**A:** Given the fact that the wave function decays rapidly after the classical turning points $x_-$ and $x_+$, we can choose the $x_\mathrm{lower} << x_-$ and  $x_\mathrm{upper} >> x_+$, and set the boundary conditions as 

$$\psi(x_\mathrm{lower})=0$$
$$\psi(x_\mathrm{upper})=0$$

**Q:** Whether we start the propogation from the left or the right boundary, we only know $\psi(x_0)$, but the propogation needs two known points $\psi(x_0)$ and $\psi(x_1)$ to get the 3rd point $\psi(x_2)$.

**A:** We can choose an arbitary positive value for $\psi(x_1)$ (and  $\psi(x_{n-1})$), and later we can normalize the wave funciton.

In [1]:
%matplotlib widget
import ipywidgets as widgets
import numpy as np
import matplotlib.pyplot as plt 
import time

hbar = 1
class NumerovSolverHO:
    def __init__(self,m,omega, xlower, xupper, npoints=1000):
        self.m = m
        self.omega = omega
        self.xlower = xlower
        self.xupper = xupper
        self.npoints = npoints
        self.x = np.linspace(self.xlower,self.xupper,self.npoints)
        self.delta = self.x[1]-self.x[0]
        self.x1 = 1e-2 # A small positive value as the initial guess the psi value at the second grid point
        self.psi_left = None
        self.psi_right = None
        self.prob_left = None
        self.prob_right = None
        self.node = 0
        self.sign = 1
    
    def getV(self, x):
        return 0.5*self.m*self.omega**2*x**2
        
    def getK2(self, E, x):
        return 2*self.m/(hbar**2)*(E - self.getV(x))
    
    def propogateNumerov(self, E, x0, x1, psi0, psi1):
        psi2 = 2*psi1 - psi0 - self.getK2(E,x1)*psi1*self.delta**2
        return psi2
    
    def Numerov_left(self, E):
        self.psi_left = np.zeros(len(self.x))
        self.psi_left[1] = self.x1
        for i in range(1,len(self.x)-1):
            self.psi_left[i+1] = self.propogateNumerov(E, self.x[i-1],self.x[i],self.psi_left[i-1],self.psi_left[i])
        leftTP,rightTP = self.find_turningPts(E)
        self.prob_left = np.trapz(np.power(self.psi_left[max(0,leftTP):min(rightTP+1,self.npoints)],2),self.x[max(0,leftTP):min(rightTP+1,self.npoints)])
        self.psi_left_normed = self.psi_left/np.sqrt(self.prob_left)
        
    def Numerov_right(self, E):
        self.psi_right = np.zeros(len(self.x))
        self.psi_right[-2] = self.x1
        for i in range(len(self.x)-2,0,-1):
            self.psi_right[i-1] = self.propogateNumerov(E, self.x[i+1],self.x[i],self.psi_right[i+1],self.psi_right[i])
        leftTP,rightTP = self.find_turningPts(E)
        self.prob_right = np.trapz(np.power(self.psi_right[max(0,leftTP):min(rightTP+1,self.npoints)],2),self.x[max(0,leftTP):min(rightTP+1,self.npoints)])
        self.psi_right_normed = self.psi_right/np.sqrt(self.prob_right)
        
    def find_turningPts(self, E):
        # Left
        leftTP = 0
        for i in range(0,self.npoints):
            if self.getV(self.x[i])<E:
                leftTP = i
                break


        rightTP = self.npoints-1
        for i in range(self.npoints-1,0,-1):
            if self.getV(self.x[i])<E:
                rightTP = i
                break  
        return leftTP,rightTP
    
    def calc(self,E):
        self.Numerov_left(E)
        self.Numerov_right(E)

        # Find the turning points to be later used as matching points
        leftTP,rightTP=self.find_turningPts(E)
        
        i_matching = rightTP
        # Scale to make psi_left and psi_right have the same sign
        self.psi_right *= self.psi_left[i_matching]/self.psi_right[i_matching]
        self.psi_right_normed *= self.psi_left_normed[i_matching]/self.psi_right_normed[i_matching]

        #Mactching condition
        F = (self.psi_left[i_matching+1]-self.psi_left[i_matching-1]) - (self.psi_right[i_matching+1]-self.psi_right[i_matching-1])
        F = F/(2*self.delta)/self.psi_left[i_matching]

        # Count the number of nodes in psi
        nnode_curr = 0
        for i in range(0,i_matching):
            if self.psi_left[i]*self.psi_left[i+1] < 0:
                nnode_curr += 1

        # When a new node develops in psi, we need to flip the sign of F
        if nnode_curr != self.node:
            self.sign = - self.sign
            self.node = nnode_curr

        F = F*self.sign

        return F

    def find_roots(self,E,F):
        roots = []
        for i in range(0,len(E)-1):
            if F[i]*F[i+1] < 0:
                roots.append(0.5*(E[i]+E[i+1]))
        return roots
    
    def solve_E(self, Emin,Emax,nE):
        self.E_list = np.linspace(Emin,Emax,nE)
        F_list = []
        self.node = 0
        self.sign = 1
        for E in self.E_list:
            F = self.calc(E)
            F_list.append(F)

        self.F_list = np.array(F_list)

        self.E_roots = self.find_roots(self.E_list,F_list)

        print("Numerical eigenvalues:", self.E_roots)
        
    def plotWFN(self, E):
        figure=plt.figure()
        F = self.calc(E)
        plt.plot(self.x,self.psi_left,label=r'$\psi_\mathrm{left}$')
        plt.plot(self.x,self.psi_right,label=r'$\psi_\mathrm{right}$')
        plt.title("E= " + '{:10.4f}'.format(E) + "\nMatching Error: " + '{:10.6f}'.format(F))
        plt.legend()
        plt.xlabel(r'$x$(bohr)')
        plt.ylabel(r'$\psi(x)$')
        leftTP,rightTP=self.find_turningPts(E)
        plt.ylim(min(self.psi_left[leftTP:rightTP+1]-1),max(self.psi_left[leftTP:rightTP+1])+1)
        plt.xlim(self.x[leftTP],self.x[rightTP])


## Run the following code to see how the choice of boundaries can change the Numerov solution.

For simplicity, we assume that $m=0.5$, $\omega=2$, and we know that $E=1.0$ is the energy of the lowest state. We fix the number of discrete points to be 1000 in the range

In [2]:
m1 = 0.5
omega1 = 2
npoints=1000
E1 = 1.0

# Upper boundary for the range of x to be solved by Numerov.  xupper can take value between 1 and 10 (bohr) inputed by the user, just for demonstration purpose.
xupper_options = widgets.FloatSlider(
    description=r'$x_\mathrm{upper}$:',                             
    value=5.0,
    min=1.0,
    max=10.0,
    step=0.1,
    padding = 8)

@widgets.interact(xupper=xupper_options)
def update(xupper):
    plt.clf()
    solver=NumerovSolverHO(m1,omega1,-xupper,xupper,npoints)
    start=time.time()
    solver.calc(E1)
    end=time.time()
    timetaken=end-start
    
    plt.title("Numerov solution to HO, "
              + "\nTime taken: {:.2f} ms".format(timetaken*1000))
    
    # plot both the lines and points to make it visually better
    plt.plot(solver.x, solver.psi_left_normed, c='b',label=r'$\psi_\mathrm{left}$')
    plt.xlim(-10,10)
    plt.ylim(-1,1)
    
    plt.plot(solver.x, solver.psi_right_normed, c='r',label=r'$\psi_\mathrm{right}$')
    plt.xlabel(r'$x$ (bohr)')
    plt.ylabel(r'$\psi(x)$')
    plt.legend()



interactive(children=(FloatSlider(value=5.0, description='$x_\\mathrm{upper}$:', max=10.0, min=1.0), Output())…

## Next, let's explore how the wavefunction shape change with E (what if E is not an eigenvalue)

In [5]:
# Upper boundary for the range of x to be solved by Numerov.  xupper can take value between 1 and 10 (bohr) inputed by the user, just for demonstration purpose.
E_options = widgets.FloatSlider(
    description=r'$E$:',                             
    value=0.1,
    min=0.1,
    max=5.0,
    step=0.1,
    padding = 8)

m1 = 0.5
omega1 = 2
npoints=1000
xupper=10

fig2=plt.figure()
@widgets.interact(E=E_options)
def update2(E):
    plt.clf()
    solver=NumerovSolverHO(m1,omega1,-xupper,xupper,npoints)
    start=time.time()
    solver.calc(E)
    end=time.time()
    timetaken=end-start
    
    plt.title("Numerov solution to HO, "
              + "\nTime taken: {:.2f} ms".format(timetaken*1000))
    
    # plot both the lines and points to make it visually better
    plt.plot(solver.x, solver.psi_left_normed, c='b',label=r'$\psi_\mathrm{left}$')
    plt.xlim(-10,10)
    plt.ylim(-1,1)
    
    leftTP,rightTP=solver.find_turningPts(E)
    plt.plot([solver.x[leftTP],solver.x[leftTP]],[-1,1],c='gray')
    plt.plot([solver.x[rightTP],solver.x[rightTP]],[-1,1],c='gray')
    plt.text(solver.x[rightTP],0.5,'Classical turning points')
    
    plt.plot(solver.x, solver.psi_right_normed, c='r',label=r'$\psi_\mathrm{right}$')
    plt.xlabel(r'$x$ (bohr)')
    plt.ylabel(r'$\psi(x)$')
    plt.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(FloatSlider(value=0.1, description='$E$:', max=5.0, min=0.1), Output()), _dom_classes=('…

In [148]:
solver.solve_E(0.1,4,1000)

Numerical eigenvalues: [0.9998498498498498, 2.9986486486486488, 3.346096096096096]


# Now let's see how the matching error changes with $E$

Plot the matching error $\psi_\mathrm{left}(x_+)-\psi_\mathrm{right}(x_+)$ as a funciton of $E$

In [8]:
m1 = 0.5
omega1 = 2
npoints=1000
xupper=10


Emin=0.1
Emax=5.0
nE = 500

solver=NumerovSolverHO(m1,omega1,-xupper,xupper,npoints)

start=time.time()
solver.solve_E(Emin,Emax,nE)
end = time.time()
timetaken = end-start

fig3=plt.figure()
plt.plot(solver.E_list,solver.F_list,c='b')
plt.plot([solver.E_list[0],solver.E_list[-1]],[0,0],c='k')

Fmin = min(solver.F_list)
Fmax = max(solver.F_list)
    
for root in solver.E_roots:
    plt.plot([root,root],[Fmin,Fmax],c='r')
plt.xlabel(r'$E(E_h)$')
plt.ylabel('Matching Error')

plt.title("Matching error as a function of E, "
          + "\nTime taken: {:.2f} ms".format(timetaken*1000))


Numerical eigenvalues: [0.998496993987976, 3.0017034068136272, 4.995090180360721]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Matching error as a function of E, \nTime taken: 3828.73 ms')

## The E values corresponds to 0 matching error are the eigenvalues. Therefore, we can find them as the points where the error function F(E) changes sign.